# 航拍仙人掌预测 

In [1]:
{
    print("Author: EugeneLiu")
}

Author: EugeneLiu


{None}

## 导入需要的 

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)
/usr/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [3]:
# print(os.listdir("./"))

## 文件路径设置和train文件的读取 

In [4]:
train_dir = "./train"
test_dir = "./test"

train_csv = pd.read_csv("./train.csv")

In [5]:
# train.head(5)

In [6]:
# type(train.has_cactus[0])

In [7]:
# train.shape

In [8]:
# train.has_cactus.value_counts()

In [9]:
# 预备对象待时机写入到TFRecord文件中
# tfrecord_filename = ("train_data-000")
# writer = tf.python_io.TFRecordWriter("./tfrecord/"+tfrecord_filename)

In [10]:
# 将数据写入到TFRecord中
# def create_record():
#     for row_number,row in train.iterrows():
# #         print(row["id"]+"  "+str(row["has_cactus"]))
#         imgpath =train_dir + "/" + row["id"]
# #         print(imgpath)
#         img = Image.open(imgpath,"r")
#         img_raw = img.tobytes()
# #         print(kind+"   "+img_raw)
#         examples = tf.train.Example(
#             features=tf.train.Features(
#                 feature={
#                     "label" : tf.train.Feature(int64_list=tf.train.Int64List(value=[row["has_cactus"]])),
#                     "img_raw" : tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
#                 }
#             )
#         )
# #         print(img_raw)
#         writer.write(examples.SerializeToString())
#     writer.close()
        
# create_record()        
# #        print(filepath)

In [11]:
# def load_data_and_decode_data():
#     reader = tf.TFRecordReader()
#     _,serialized_example = reader.read(tf.train.string_input_producer(["./tfrecord/train_data-000"]))
#     features = tf.parse_single_example(serialized_example,
#                                       features={
#                                               "label":tf.FixedLenFeature([],tf.int64),
#                                               "img_raw":tf.FixedLenFeature([],tf.string)
#                                       })
#     label = features["label"]
#     img = features["img_raw"]
# #     print(type(label))
# #     print(type(img))
#     img = tf.decode_raw(img,tf.uint8)
#     img = tf.reshape(img,[32,32,3])
#     img = tf.cast(img,tf.float32) * (1. / 255)
#     label = tf.cast(label,tf.float32)
#     return img,label

In [12]:
# batch = load_data_and_decode_data()

In [13]:
# init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())

In [14]:
# train.shape[0]

In [15]:
# with tf.Session() as sess:
#     sess.run(init_op)
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord = coord)
#     for i in range(train.shape[0]):
#         example,label = sess.run(batch)
# #         print(example)
# #         print(label)
#         img = Image.fromarray(example,'RGB')
#         img.save("./processedPictures/"+str(i)+"_"+ str(int(label)) +".jpg")
#     coord.request_stop()
#     coord.join(threads)
#     sess.close()

## 分验证集和测试集

In [16]:
# from sklearn.model_selection import train_test_split

In [17]:
# data = []
# label = []
# for imgPath in os.listdir("./processedPictures"):
#     data.append("./processedPictures/"+imgPath)
#     label.append(imgPath.split("_")[1].split(".")[0])train
#     print(imgPath.split("_")[1].split(".")[0])

In [18]:
# data_train,data_validation,label_train,label_validation = train_test_split(data,label,test_size=0.3,random_state=0)

In [19]:
# data_train[:10]

In [20]:
# data_validation[:10]

In [21]:
# label_train[:10]

In [22]:
# label_validation[:10]

In [23]:
# len(data_train)

## 使用tensorflow搭建卷积神经网络 

In [24]:
INPUT_NODE = 32 * 32 * 3 #3072
OUTPUT_NODE = 2

IMAGE_SIZE =32
NUM_CHANNELS = 3#RGB为三个维度,图层数目为1
NUM_LABELS = 1

CONV1_DEEP = 6
CONV1_SIZE = 5
CONV1_STEP = 1

CONV2_DEEP = 16
CONV2_SIZE = 5
CONV2_STEP = 1

FC_SIZE = 120 #自己设置第五层的全连接层的节点数目
FC2_SIZE = 84 #自己设置第六层的全连接层的节点数目
FC3_SIZE = 2

In [25]:
def forward_propagation(input_tensor,is_train,regularizer,batch):#一次处理的图片,批处理大小,可预测种类数
    print(input_tensor)
    with tf.variable_scope("conv1"):#名字为conv1,看打印日志的时候需要看这个
        conv1_weights = tf.get_variable("weight",[CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP],
                              initializer = tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable("bias",[CONV1_DEEP],initializer = tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='VALID')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
    with tf.name_scope("pool1"):
        pool1 = tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
#         return pool1
    with tf.variable_scope("conv2"):
        #定义第二层网络结构
        conv2_weights = tf.get_variable("weight",[CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP],
                                   initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias",[CONV2_DEEP],initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding="VALID")
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
    with tf.name_scope("pool2"):
        pool2 = tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
            
    pool_shape = pool2.get_shape().as_list()
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
#     return pool_shape[0]
#     print(pool_shape[0])
    reshaped = tf.reshape(pool2,[batch,nodes])
    with tf.variable_scope("fc1"):
        fc1_weights = tf.get_variable("weight",[nodes,FC_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer != None):
             tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias",[FC_SIZE],initializer=tf.constant_initializer(0.1))
        fc1 = tf.nn.relu(tf.matmul(reshaped,fc1_weights) + fc1_biases)
#         if(is_train):
#             fc1 = tf.nn.dropout(fc1,0.5) #0.5的概率将这个点的值变成0
    with tf.variable_scope("fc2"):
        fc2_weights = tf.get_variable("weight",[FC_SIZE,FC2_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer != None):
            tf.add_to_collection('losses',regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias",[FC2_SIZE],initializer=tf.constant_initializer(0.1))
        fc2 = tf.nn.relu(tf.matmul(fc1,fc2_weights) + fc2_biases)
#         if(is_train):
#             fc2 = tf.nn.dropout(fc2,0.5)
    with tf.variable_scope("fc3"):
        fc3_weights = tf.get_variable("weight",[FC2_SIZE,FC3_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer != None):
            tf.add_to_collection('losses',regularizer(fc3_weights))
        fc3_biases = tf.get_variable("bias",[FC3_SIZE],initializer=tf.constant_initializer(0.1))
        fc3 = tf.nn.relu(tf.matmul(fc2,fc3_weights) + fc3_biases)  
        return fc3

In [26]:
BATCH_SIZE = 175
LEARNING_BASE_RATE = 0.8   #基础的学习率
LEARNING_RATE_DECAY = 0.99 #学习率的衰减率
REGULARIZATION_RATE = 0.0001
EPOCHS = 1000
MOVING_AVERAGE_DECAY = 0.99

BATCH_SIZE = 175

MODEL_SAVE_PATH = "./model"
MODEL_NAME = "model.ckpt"

In [27]:
def train(feed_data,feed_label,batch_size):
    x_train = tf.placeholder(tf.float32,[None,32,32,3],name="x-input")
    y_train = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-input')
    
#     dataset = tf.data.Dataset.from_tensor_slices((feed_data,feed_label))
#     print(dataset)
#     dataset = dataset.shuffle(20).batch(175).repeat()
#     iterator = dataset.make_initializable_iterator()
#     data_element = iterator.get_next()
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
#     return forward_propagation(x_train,True,regularizer)
    y_predict = forward_propagation(x_train,True,regularizer,batch_size)
#     print(y_predict)
#     a = tf.Variable(y_predict,name="y-predict")
#     return y_predict
    
    training_times = tf.Variable(0,trainable=False)
    #滑动平均
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,training_times)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    #损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits = y_predict,
        labels = tf.argmax(y_train,1)
    )#tf.argmax(y_train,1) 将one_hot编码格式下的数据转换为对应的label
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection("losses"))
    #学习率
    learning_rate = tf.train.exponential_decay(LEARNING_BASE_RATE,training_times,len(data_train)/BATCH_SIZE,LEARNING_RATE_DECAY)
    #梯度下降法优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,training_times)
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op = tf.no_op(name = 'train')
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
#         sess.run(dataset,feed_dict = {
#             x_train:feed_data,
#             y_train:feed_label
#         })
#         sess.run(iterator.initializer)
        for epoch in range(EPOCHS):            
#             x_batch,y_batch = sess.run(data_element)
            for i in range(69):
                x_batch = feed_data[i * batch_size : batch_size * (i + 1)]
                y_batch = feed_label[i * batch_size : batch_size * (i + 1)]
                _, loss_value, step = sess.run([train_op,loss,training_times],feed_dict={
                    x_train:x_batch,
                    y_train:y_batch
                })
            if(epoch % 10 == 0) :
                print("经过了"+str(epoch + 1)+"次训练之后,每个训练批次中损失函数的值为"+str(loss_value))
                saver.save(sess,"./model/model_"+str(epoch)+".ckpt",training_times)
                get_score()
                if(epoch == 0):
                    return
                

In [87]:
def get_score():
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32,[5250,32,32,3],name="x-input")
        y = tf.placeholder(tf.float32,[5250,2],name = "y-input")
        validate_feed = {
            x: np.array(feed_data_validation),
            y: np.array(feed_label_validation)
        }
        predict = forward_propagation(x,False,None,5250)
    
        
        correct_prediction = tf.equal(tf.argmax(predict,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state("model/")
            saver = tf.train.import_meta_graph(ckpt.model_checkpoint_path+".meta")    

            saver.restore(sess, ckpt.model_checkpoint_path)
            accuracy_score = sess.run("fc3/Relu:0",feed_dict = validate_feed)
            print(accuracy_score)
            print(sess.run(tf.argmax([0.0,0.0],1)))
            return accuracy_score
    #         graph = tf.get_default_graph()   

    
    #         input_tensor = graph.get_operation_by_name('x-input').outputs[0]
    #         feed_dict = {input_tensor.name: np.array(feed_data_test)
    #         y_predict = graph.get_operation_by_name("fc3/Relu").outputs[0]
    #         viterbi_sequence = sess.run(y_predict, feed_dict)

In [92]:
np.argmax([1,0])

0

In [95]:
predict_csv = "sample_submission.csv"

In [124]:
predict_data = pd.read_csv(predict_csv)

In [125]:
predict_data.head()

,Unnamed: 0,id,has_cactus
0,0,000940378805c44108d287872b2f04ce.jpg,0.165049
1,1,0017242f54ececa4512b4d7937d1e21e.jpg,0.164839
2,2,001ee6d8564003107853118ab87df407.jpg,0.206062
3,3,002e175c3c1e060769475f52182583d0.jpg,0.215506
4,4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.164074


In [93]:
!ls

仙人掌识别.ipynb      model		 sample_submission.csv	train
仙人掌test部分.ipynb  prediction.zip	 test			train.csv
input_data	      processedPictures  tfrecord


In [72]:
type(get_score())

Tensor("x-input:0", shape=(5250, 32, 32, 3), dtype=float32)
INFO:tensorflow:Restoring parameters from model/model_372.ckpt-25737


numpy.ndarray

In [71]:
for i in np.array(get_score()):
    print(i)

Tensor("x-input:0", shape=(5250, 32, 32, 3), dtype=float32)
INFO:tensorflow:Restoring parameters from model/model_362.ckpt-25047
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]

[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]


[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]


## 分验证集和测试集

In [28]:
# data_train,label_train,data_validation,label_validation
train_csv['data'] = train_csv.apply(lambda x:np.array(Image.open("train/"+x['id'])).reshape([32,32,3]) / 255.0,axis = 1)#按行读

In [29]:
data_train,data_validation,label_train,label_validation = train_test_split(train_csv['data'],train_csv['has_cactus'],test_size = 0.3,random_state = 0)

In [30]:
feed_data_train = []
for i in data_train:
    feed_data_train.append(i)

In [31]:
feed_label_train = []
for i in label_train:
    if i == 0:
        feed_label_train.append([0,1])
    else:
        feed_label_train.append([1,0])

In [32]:
feed_data_validation = []
for i in data_validation:
    feed_data_validation.append(i)

In [33]:
feed_label_validation = []
for i in label_validation:
    if i == 0:
        feed_label_validation.append([0,1])
    else:
        feed_label_validation.append([1,0])

In [34]:
np.array(feed_data_train).shape

(12250, 32, 32, 3)

In [35]:
np.array(feed_label_train).shape

(12250, 2)

In [36]:
zero_count = 0
one_count = 0
for i in feed_label_validation:
    if(1 == i[0]):
        one_count = one_count +1
    else:
        zero_count += 1
print(str(zero_count)+"    "+str(one_count))        
print(one_count/(zero_count + one_count))

1284    3966
0.7554285714285714


In [38]:
# pool1 = 
# print(

# train(np.array(feed_data_train),np.array(feed_label_train),BATCH_SIZE)

#      )
# print(pool1)

## 对需要预测的数据 

In [39]:
!ls

仙人掌识别.ipynb      model		 sample_submission.csv	train
仙人掌test部分.ipynb  prediction.zip	 test			train.csv
input_data	      processedPictures  tfrecord


In [126]:
test_csv = pd.read_csv("./sample_submission.csv")

In [127]:
# data_train,label_train,data_validation,label_validation
test_csv['data'] = test_csv.apply(lambda x:np.array(Image.open("test/"+x['id'])).reshape([32,32,3]) / 255.0,axis = 1)#按行读

In [128]:
feed_data_test = []
for i in test_csv['data']:
    feed_data_test.append(i)

In [129]:
feed_data_test

[array([[[0.22352941, 0.23137255, 0.21176471],
         [0.21176471, 0.21960784, 0.2       ],
         [0.10588235, 0.11372549, 0.10196078],
         ...,
         [0.22352941, 0.17647059, 0.21568627],
         [0.20784314, 0.15686275, 0.19607843],
         [0.24313725, 0.19215686, 0.23137255]],
 
        [[0.20784314, 0.21568627, 0.19607843],
         [0.20392157, 0.21176471, 0.19215686],
         [0.11764706, 0.1254902 , 0.11372549],
         ...,
         [0.23529412, 0.18823529, 0.22745098],
         [0.21176471, 0.16078431, 0.2       ],
         [0.27058824, 0.21960784, 0.25882353]],
 
        [[0.38431373, 0.38431373, 0.37647059],
         [0.34901961, 0.34901961, 0.34117647],
         [0.23137255, 0.23921569, 0.22745098],
         ...,
         [0.25098039, 0.20392157, 0.24313725],
         [0.22745098, 0.17647059, 0.21568627],
         [0.2627451 , 0.21176471, 0.25098039]],
 
        ...,
 
        [[0.33333333, 0.29411765, 0.3372549 ],
         [0.32156863, 0.28235294, 0.32549

In [44]:
len(feed_data_test)

4000

In [130]:
feed_data_test = []
for i in test_csv['data']:
    feed_data_test.append(i)

In [131]:
len(feed_data_test)

4000

In [132]:
np.array(feed_data_test)

array([[[[0.22352941, 0.23137255, 0.21176471],
         [0.21176471, 0.21960784, 0.2       ],
         [0.10588235, 0.11372549, 0.10196078],
         ...,
         [0.22352941, 0.17647059, 0.21568627],
         [0.20784314, 0.15686275, 0.19607843],
         [0.24313725, 0.19215686, 0.23137255]],

        [[0.20784314, 0.21568627, 0.19607843],
         [0.20392157, 0.21176471, 0.19215686],
         [0.11764706, 0.1254902 , 0.11372549],
         ...,
         [0.23529412, 0.18823529, 0.22745098],
         [0.21176471, 0.16078431, 0.2       ],
         [0.27058824, 0.21960784, 0.25882353]],

        [[0.38431373, 0.38431373, 0.37647059],
         [0.34901961, 0.34901961, 0.34117647],
         [0.23137255, 0.23921569, 0.22745098],
         ...,
         [0.25098039, 0.20392157, 0.24313725],
         [0.22745098, 0.17647059, 0.21568627],
         [0.2627451 , 0.21176471, 0.25098039]],

        ...,

        [[0.33333333, 0.29411765, 0.3372549 ],
         [0.32156863, 0.28235294, 0.3254902 ]

In [136]:
 def predict_test():
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32,[4000,32,32,3],name="x-input")
        test_feed = {
            x : np.array(feed_data_test)
        }
        predict = forward_propagation(x,False,None,4000)
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state("model/")
            saver = tf.train.import_meta_graph(ckpt.model_checkpoint_path+".meta")    

            saver.restore(sess, ckpt.model_checkpoint_path)
            predict = sess.run(predict,feed_dict = test_feed)
            return predict

In [140]:
predict_result = predict_test()
predict_result

Tensor("x-input:0", shape=(4000, 32, 32, 3), dtype=float32)
INFO:tensorflow:Restoring parameters from model/model_1.ckpt-32


array([[0.1436439 , 0.20864826],
       [0.19366334, 0.31745663],
       [0.20055656, 0.22963947],
       ...,
       [0.16403787, 0.23532763],
       [0.1972153 , 0.2583064 ],
       [0.23532937, 0.29305074]], dtype=float32)

In [138]:
predict_result = pd.DataFrame(predict_result)
predict_result

,0,1
0,0.143644,0.208648
1,0.193663,0.317457
2,0.200557,0.229639
3,0.230582,0.230117
4,0.182422,0.248588
5,0.194191,0.210483
6,0.199658,0.201261
7,0.224948,0.196721
8,0.220054,0.235420
9,0.223331,0.280002


In [113]:
predict_re = test_csv.copy()



In [139]:
predict_re["has_cactus"] = predict_result[1]
del predict_re['data']

KeyError: 'data'

In [122]:
predict_re.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.165049
1,0017242f54ececa4512b4d7937d1e21e.jpg,0.164839
2,001ee6d8564003107853118ab87df407.jpg,0.206062
3,002e175c3c1e060769475f52182583d0.jpg,0.215506
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.164074


In [141]:
predict_re.to_csv("sample_submission.csv")

In [115]:
predict_result

0    0.165049
1    0.108109
dtype: float64

## 为了测试 

In [ ]:
# !ls

In [ ]:
# a = tf.constant([[1.0,2.0],[1.,2.]],name="a")
# b = tf.constant([2.0,3.0],name="b")
# result = tf.add(a,b,name="add3")https
# print (result)

In [ ]:
# from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# type(input_data)

In [ ]:
# mnist = input_data.read_data_sets("/temp/data",one_hot=True)
# mnist.train.next_batch(1)

In [ ]:
# data_train

In [ ]:
# label_train

In [ ]:
# tf.argmax([0,0,0,1,0],1)


In [ ]:
# data_train

In [ ]:
# np.array(Image.open(data_train[0],"r"))

## Todo:
- 解决维度错误问题  OK
  1. 删掉reshape
  2. 添加batch
- 